In [ ]:
# Activate project
using Pkg
Pkg.activate("/Users/alessiogiorlandino/Desktop/SAM/Project.toml")
Base.active_project()
# Resolve dependencies
Pkg.resolve()

In [1]:
import Rfam, Infernal, Logomaker, FASTX
using Optimisers: Adam
using EllipsisNotation: (..)
using FillArrays: Falses
import Makie, CairoMakie
using Makie: @colorant_str
import SamApp
using LinearAlgebra
using Statistics

using CSV
using DataFrames

using BioSequences: LongRNA, @rna_str
using Statistics: mean, std, cor

    CondaPkg Found dependencies: /Users/alessiogiorlandino/.julia/packages/Logomaker/DdglG/CondaPkg.toml

    CondaPkg Found dependencies: /Users/alessiogiorlandino/.julia/packages/PythonCall/dsECZ/CondaPkg.toml


    CondaPkg Found dependencies: /Users/alessiogiorlandino/.julia/packages/Logomaker/DdglG/CondaPkg.toml


    CondaPkg Found dependencies: /Users/alessiogiorlandino/.julia/packages/PythonCall/dsECZ/CondaPkg.toml


    CondaPkg Dependencies already up to date


In [31]:
MSA_df=SamApp.rf00162_hits_taxonomy()
sp_taxonomy = split.(MSA_df[:,"taxonomy"], "; ")
phyla = fill("other", length(sp_taxonomy))
for i in 1:length(sp_taxonomy)
    if length(sp_taxonomy[i]) >= 2
        phyla[i] = sp_taxonomy[i][2]  
    end
end
MSA_df[!,:phylum] = phyla
counts = sort!(combine(groupby(MSA_df, :phylum), :phylum => length => :count), :count, rev=true)


Row,phylum,count
,String,Int64
1,Bacillota,3679
2,Bacteroidota,854
3,Actinomycetota,691
4,Pseudomonadota,187
5,Chloroflexi,142
6,Firmicutes,84
7,Fusobacteria,43
8,Deinococcus-Thermus,38
9,other,33


In [49]:
for j in 1:nrow(MSA_df)
    if counts.count[findall(counts.phylum.== MSA_df[j, :phylum])][1] < 150 
        MSA_df[!, :phylum][j] = "other"
    end
end

label_file = open("labels.txt","w+")
#write(label_file, "#$(counts.phylum)\n")
#labels: ["Bacillota", "Bacteroidota", "Actinomycetota", "other", "Pseudomonadota", "Chloroflexi"]
for label in MSA_df.phylum
    println(label_file, label)
end
close(label_file)